In [6]:
import re
import pandas as pd


# 文件路径
file_path = '/mnt/c/Users/victo/Desktop/19.xml'

# 读取文件内容
with open(file_path, 'r', encoding='ISO-8859-1') as file:
    content = file.read()

# 使用正则表达式提取目标内容
pattern = r'Label\s+Population name.*?</pairDistPopLabels>\s*<data>'
match = re.search(pattern, content, re.DOTALL)

# 获取并清理匹配到的内容
if match:
    extracted_content = match.group()
else:
    extracted_content = "未找到匹配内容"


# 将提取的内容进行行分割
lines = extracted_content.splitlines()

# 跳过标题行，提取有效数据
data = []
for line in lines:
    if line.startswith("  "):  # 数据行以两个空格开头
        parts = line.split(":\t")
        if len(parts) == 2:
            label, population_name = parts
            data.append((label.strip(), population_name.strip()))

# 创建DataFrame
df_list = pd.DataFrame(data, columns=["Label", "Population_Name"])

# 使用正则表达式提取目标内容
pattern_fst = r'Computing conventional F-Statistics from haplotype frequencies.*?</PairFstMat>\s*<data>'
match_fst = re.search(pattern_fst, content, re.DOTALL)

# 获取并清理匹配到的内容
if match_fst:
    extracted_fst_content = match_fst.group()
else:
    extracted_fst_content = "未找到匹配内容"

extracted_fst_content[:1000]  # 仅显示前1000个字符以便查看结果

# 分割提取的内容为行
lines_fst = extracted_fst_content.splitlines()

# 提取列名称和数据部分
header_line = lines_fst[1].strip() if len(lines_fst) > 1 else ""
data_lines = lines_fst[2:]

# 提取列名（列名为数字序列）
columns = header_line.split()

# 解析每一行的数字数据
fst_data = []
for line in data_lines:
    if line.strip():  # 跳过空行
        row = line.split()
        fst_data.append(row)

# 动态处理行长度不一致的问题
from itertools import zip_longest

# 确保每一行的长度与列数量匹配
max_columns = max(len(line.split()) for line in data_lines)
formatted_data = [line.split() for line in data_lines]
normalized_data = list(zip_longest(*formatted_data, fillvalue=None))

# 确保列数与最长的行对齐
columns_adjusted = [str(i + 1) for i in range(len(normalized_data))]

# 转换为DataFrame
df_fst_fixed = pd.DataFrame(normalized_data).T
df_fst_fixed.columns = columns_adjusted
df_fst_fixed = df_fst_fixed.rename(columns={'1': 'Label'})


df_combine = df_fst_fixed.merge(df_list, on='Label', how='left')
df_combine['Label'] = df_combine['Population_Name']
df_combine = df_combine.drop(axis=1,columns='Population_Name')
df_combine.drop(axis=0,index=max_columns-1)

df_combine.to_csv('/mnt/c/Users/victo/Desktop/19.csv',index=False,header=False)